In [1]:
import cv2
import numpy as np
import math


# ## Đọc ảnh input, grayscale, Otsu và Binary threshold
image = cv2.imread('table7.jpg') #Đọc hình ảnh
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #chuyển ảnh thành xám

thresh, img_bin = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # xác định giá trị ngưỡng trong khoảng 128 đến 255 bằng Binary và Ostu
img_bin = 255-img_bin #Chuyển đổi ảnh binary dựa trên ngưỡng tìm được.

# Tìm các đường ngang
kernel_len = gray.shape[1]//120  # tính chiều cao cho nhân (gray,shape[1] sẽ trả về chiều cao)

hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))  #dùng hình thái học yêu cầu một mô hình cấu trúc chữ nhật với kích thước là knel_len và điểm neo là 1
print(kernel_len)
image_horizontal = cv2.erode(img_bin, hor_kernel, iterations=3) #loại bỏ nhiễu cho ảnh trắng đen img_bin với kích thướng là hor_kernel và sự lặp lại là 3.
horizontal_lines = cv2.dilate(image_horizontal, hor_kernel, iterations=3) # dùng cv2.dilate để giãn nở ma trận ảnh với nhân là hor_kernel sự lặp lại bằng 3.
h_lines = cv2.HoughLinesP(horizontal_lines, 1, np.pi/180, 30, maxLineGap=250) #dùng để phát hiện đường thẳng viền của bảng bằng giải thuật hough transform.




# ## Nhóm các đường ngang nếu chúng ở cùng một dòng, chúng ta sẽ thu được những đường thẳng cạnh cần thiết
#
# ### Input: danh sách các đường ngang, ngưỡng của các dòng (cách nhau bao xa)- h_lines, threshold_thin
# ### Output: các đường ngang mới đã được nhóm
def group_horizontal(h_lines, threshold_thin):
    new_h_lines = []
    while len(h_lines) > 0:
        thresh = sorted(h_lines, key=lambda x: x[0][1])[0][0]
        lines = [line for line in h_lines if thresh[1] - threshold_thin <= line[0][1] <= thresh[1] + threshold_thin]
        h_lines = [line for line in h_lines if thresh[1] - threshold_thin > line[0][1] or line[0][1] > thresh[1] + threshold_thin]
        x = []
        for line in lines:
            x.append(line[0][0])
            x.append(line[0][2])
        x_min, x_max = min(x) - int(5*threshold_thin), max(x) + int(5*threshold_thin)
        new_h_lines.append([x_min, thresh[1], x_max, thresh[1]])
    return new_h_lines
    
new_horizontal_lines = group_horizontal(h_lines, kernel_len)

# ## Tìm các đường dọc tương tự như tìm đường ngang ở trên
kernel_len = gray.shape[1]//120
ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
image_vertical = cv2.erode(img_bin, ver_kernel, iterations=3)
vertical_lines = cv2.dilate(image_vertical, ver_kernel, iterations=3)
v_lines = cv2.HoughLinesP(vertical_lines, 1, np.pi/180, 30, maxLineGap=250)


# ## Nhóm các đường dọc nếu chúng ở cùng một dòng, chúng ta sẽ thu được những đường thẳng cạnh cần thiết
#
# ### Input: danh sách các đường dọc, ngưỡng của các dòng (cách nhau bao xa)- v_lines, threshold_thin
# ### Output: các đường dọc mới đã được nhóm
def group_vertical(v_lines, threshold_thin):
    new_v_lines = []
    while len(v_lines) > 0:
        thresh = sorted(v_lines, key=lambda x: x[0][0])[0][0]
        lines = [line for line in v_lines if thresh[0] -
                 threshold_thin <= line[0][0] <= thresh[0] + threshold_thin]
        v_lines = [line for line in v_lines if thresh[0] - threshold_thin >
                   line[0][0] or line[0][0] > thresh[0] + threshold_thin]
        y = []
        for line in lines:
            y.append(line[0][1])
            y.append(line[0][3])
        y_min, y_max = min(y) - int(4*threshold_thin), max(y) + int(4*threshold_thin)
        new_v_lines.append([thresh[0], y_min, thresh[0], y_max])
    return new_v_lines
    
new_vertical_lines = group_vertical(v_lines, kernel_len)

# ## Tìm điểm giao nhau của một đường ngang và một đường dọc
# 
# ### Input: hai đường thẳng (một đường ngang và một đường dọc của bảng)
# ### Output: điểm giao nhau của hai đường input
def get_intersect(line1: list, line2: list):
    a1, a2 = line1
    b1, b2 = line2
    da = a2-a1
    db = b2-b1
    dp = a1-b1

    def perp(a):
        b = np.empty_like(a)
        b[0] = -a[1]
        b[1] = a[0]
        return b

    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)
    return (num / denom.astype(float))*db + b1


# ## Tìm các điểm giao nhau
points = []
for hline in new_horizontal_lines:
    x1A, y1A, x2A, y2A = hline
    for vline in new_vertical_lines:
        x1B, y1B, x2B, y2B = vline
        line1 = [np.array([x1A, y1A]), np.array([x2A, y2A])]
        line2 = [np.array([x1B, y1B]), np.array([x2B, y2B])]

        x, y = get_intersect(line1, line2)
        if x1A <= x <= x2A and y1B <= y <= y2B:
            points.append([int(x), int(y)])



# ## Gaussian blur, Otsu's threshold
thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] # tạo 1 ngưỡng mới cho ảnh xám gray từ 0 đến 255
# ## Tạo phần tử cấu trúc hình chữ nhật và dilate
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))#mô hình cấu trúc chữ nhật với kích thước là 5x5 
dilate = cv2.dilate(thresh1, kernel, iterations=4) #giãn nở ma trận thresh1 với nhân là kernel và 


# ## Tìm các contours, vẽ hình chữ nhật và số của đoạn văn và bảng
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # xác định các vector viền.
cnts = cnts[0] if len(cnts) == 2 else cnts[1] #xác định viền bên ngoài nếu như chiều dài vector các điểm = 2
para_c=0
table_c=0
## cho vòng lặp c chạy trong cnts hợp các điều kiện để đếm số đoạn text và số bảng.

for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    isTable=False
    for p in points:
        if (x <= p[0] <= x+w and y <= p[1] <= y+h):
            isTable=True 
    if (w>50 and h>50):
        if (isTable==True):
            cv2.rectangle(image, (x+5, y+5), (x + w -5, y + h -5), (0, 0, 255), 2) #vẽ viền chữ nhật bao phủ bảng
            cv2.putText(image, "Table "+str(table_c), (x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)# chèn text đếm số bảng
            table_c+=1 #số bảng trừ đi 1 (bởi vì sẽ vẽ từ dưới lên)
        else:
            cv2.rectangle(image, (x+5, y+5), (x + w -5, y + h -5), (36,255,12), 2) #vẽ viền chữ nhật bao phủ đoạn văn bản
            cv2.putText(image, "Paragraph "+str(para_c), (x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (32, 158, 68), 1, cv2.LINE_AA) # chèn text đếm số đoạn văn bản
            para_c+=1

print("Number of paragraph: ",para_c)
print("Number of table: ",table_c)
cv2.imshow('image', image)
cv2.imwrite('detect.jpg',image)
cv2.waitKey(0)
cv2.destroyAllWindows()








10
Number of paragraph:  6
Number of table:  2
